In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
"""Imported Modeules"""
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from transformers import BertTokenizer, BertForSequenceClassification
import keras

In [7]:
"""Converting data into DataFrame"""
train = pd.read_csv('/content/drive/MyDrive/nlp-getting-started/train.csv')

In [8]:
"""Dropping unnecessary columns"""
train = train.drop('location',axis=1)
x_train = train.drop('keyword',axis=1)

In [9]:
"""Shuffling the Dataset"""
x_train = x_train.sample(frac=1)

In [10]:
"""Creating Target variable"""
y_train = x_train['target']
x_train = x_train.drop('target',axis=1)

In [11]:
"""Importing BERT tokenizer and BERT model"""
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
"""Converting data into list so that we can feed it into tokenizer"""
train_texts = x_train['text'].tolist()
y_train = y_train.tolist()

In [13]:
train_texts[0]

'The Adventures of Mineness #9   The Famine Is Over http://t.co/spYmIQNeCj'

In [14]:
"""Converting data into tokens"""
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

In [15]:
"""Defining the Optimizers"""
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=6e-6)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
"""Preparing the dataset to load it into BERT model"""
import torch
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        assert len(self.encodings['input_ids']) == len(self.labels), "Encodings and labels must have the same length"

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = TextDataset(train_encodings, y_train)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)



In [17]:
train_dataset[0:2]

{'input_ids': tensor([[  101,  1996,  7357,  1997,  3067,  2791,  1001,  1023,  1996, 15625,
           2003,  2058,  8299,  1024,  1013,  1013,  1056,  1012,  2522,  1013,
           8645,  4328,  4160,  2638,  2278,  3501,   102,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [  101,  1030,  2205,  3597, 11927,  7716,  2094,  4931,  1030, 12986,
           2497, 21926,  2292,  1005,  1055,  4119,  2059,  2000,  1037, 11352,
           6415,  1056, 15472,  2674,  1012,  3453,  2202,  2035,  1012,   102,
              0,     0,     0,     0,     0,     0,     0,     0,   

In [20]:
"""Training the BERT model using the dataset"""
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    warmup_steps=500,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    lr_scheduler_type='linear',
    report_to='none'
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    optimizers=(optimizer,None),

)



In [21]:
trainer.train()

Step,Training Loss
100,0.642700
200,0.549800
300,0.455600
400,0.389700


TrainOutput(global_step=476, training_loss=0.4887931246717437, metrics={'train_runtime': 402.879, 'train_samples_per_second': 75.586, 'train_steps_per_second': 1.181, 'total_flos': 1314511054799040.0, 'train_loss': 0.4887931246717437, 'epoch': 4.0})

Testing Model using Test Dataset

In [22]:
test = pd.read_csv('/content/drive/MyDrive/nlp-getting-started/test.csv')

In [23]:
id = test['id']

In [24]:
test = test.drop('location',axis=1)
x_test = test.drop('keyword',axis=1)

In [25]:
x_test = x_test.drop('id',axis=1)

In [26]:
new_text = x_test['text'].tolist()

In [27]:
new_encodings = tokenizer(new_text, truncation=True, padding=True)


class PredictDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [28]:
predict_dataset = PredictDataset(new_encodings)
predict_loader = DataLoader(predict_dataset, batch_size=8, shuffle=False)

In [29]:
"""Predicting using BERT"""
model.eval()
predictions = []
with torch.no_grad():
    for batch in predict_loader:
        inputs = {key: val.to(model.device) for key, val in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predictions.append(logits)

In [30]:
predictions = torch.cat(predictions)
predicted_labels = torch.argmax(predictions, dim=1)

In [31]:
pre = pd.DataFrame(predicted_labels.cpu().numpy()) # Move the tensor to CPU before converting to NumPy array

In [32]:
predict = pd.concat((id,pre),axis=1)

In [33]:
predict = predict.rename(columns={0:'target'})

In [34]:
predict[10:20]

,id,target
10,30,0
11,35,0
12,42,0
13,43,0
14,45,0
15,46,1
16,47,0
17,51,1
18,58,0
19,60,0


In [35]:
predict.to_csv("/content/drive/MyDrive/Kaggle/predictBr6.csv",index=False)


In [1]:
!pip install torch-optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [2]:
!pip uninstall -y accelerate transformers


Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2


In [3]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 30.7 MB/s eta 0:00:00


In [4]:
!pip install transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
d = {'id':[[1,2],[3,4]],'text':['hi','hello'],'target':[0,1]}

In [ ]:
d[0]

KeyError: 0